In [0]:
# Célula para limpeza (execute uma vez)
try:
    dbutils.fs.unmount("/mnt/ecommerce")
    print("Ponto de montagem /mnt/ecommerce removido.")
except:
    print("Nenhum ponto de montagem /mnt/ecommerce encontrado para remover.")

In [0]:
# --- Configurações ---
storage_account_name = "dlsecommerceportfolio" 

# Nomes do Secret Scope e da chave que você configurou no Databricks/Key Vault.
secret_scope_name = "kv-ecommerce-scope"  # <-- SUBSTITUA pelo nome do seu scope
secret_key_name = "datalake-access-key"     # <-- SUBSTITUA pelo nome do seu secret

try:
    storage_account_key = dbutils.secrets.get(scope=secret_scope_name, key=secret_key_name)
    print("Chave de acesso recuperada com sucesso do Key Vault.")
except Exception as e:
    print(f"Erro ao buscar o segredo do Key Vault: {e}")
    # Interrompe a execução se a chave não puder ser obtida
    dbutils.notebook.exit("Falha ao obter credenciais do Key Vault.")
# --- Loop para Montar cada Container ---
for container in containers_para_montar:
    mount_point = f"/mnt/{container}"
    source_url = f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/"

    # Primeiro, verificamos se o ponto de montagem já não existe
    if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        try:
            # Se não existe, tentamos montar
            dbutils.fs.mount(
                source = source_url,
                mount_point = mount_point,
                extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
            )
            print(f"SUCESSO: Container '{container}' foi montado em '{mount_point}'")
        except Exception as e:
            print(f"ERRO ao montar o container '{container}': {e}")
    else:
        # Se já existe, apenas informamos
        print(f"AVISO: O ponto de montagem '{mount_point}' já existe. Nenhuma ação necessária.")

print("\nVerificação final dos pontos de montagem:")
# Exibe todos os pontos de montagem ativos para confirmação
display(dbutils.fs.mounts())